In [23]:
import torch

In [25]:
torch

<module 'torch' from 'C:\\ProgramData\\anaconda3\\Lib\\site-packages\\torch\\__init__.py'>

# 反向传播的一些理解
 反向传播的深入理解 https://gengzhige-essay.readthedocs.io/docs/03%20%E6%84%9F%E7%9F%A5%E6%9C%BA/3-3%20%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD%E7%9A%84%E6%B7%B1%E5%85%A5%E7%90%86%E8%A7%A3.html

[梯度下降和反向传播算法数据科学家指南](https://developer.nvidia.com/zh-cn/blog/a-data-scientists-guide-to-gradient-descent-and-backpropagation-algorithms/) 


[一文弄懂神经网络中的反向传播法——BackPropagation](https://www.cnblogs.com/charlotte77/p/5629865.html)


在反向传播过程中,每一行数据(或每一个批次的数据)都会用来更新所有的权重(weights)。这是神经网络学习的核心机制。具体来说:

1. 前向传播: 输入数据通过网络,计算预测输出。

2. 计算损失: 比较预测输出和实际标签,得到误差。

3. 反向传播: 误差从输出层向输入层反向传播,计算每个权重对误差的贡献(梯度)。

4. 更新权重: 使用优化算法(如梯度下降)根据计算出的梯度更新所有权重。

这个过程对每一行数据(在批量处理中是每一批数据)都会重复进行。因此:

- 在随机梯度下降(SGD)中,每处理一行数据就更新一次所有权重。
- 在小批量梯度下降中,每处理一个批次的数据就更新一次所有权重。
- 在批量梯度下降中,处理完所有数据后更新一次所有权重。

这种频繁的更新允许模型逐步学习数据中的模式,不断调整以最小化预测误差。

需要注意的是,虽然每次都更新所有权重,但每个权重的更新幅度可能不同,这取决于该权重对当前误差的贡献程度。

In [27]:
import torch

# Create a tensor and set requires_grad=True to track computations
x = torch.tensor(2.0, requires_grad=True)

# Define a simple function
y = x**2 + 3*x + 1

# Compute the gradients
y.backward()

# Print the gradient of y with respect to x
print(x.grad)  # This will output the gradient dy/dx

tensor(7.)


In [75]:
import torch

# Step 1: Create a tensor for input (x) and target output (y_true)
x = torch.tensor([[3.0, 2.0, 1.0],[1.3,2.4,3.7]])  # Input feature with 3 dimensions
y_true = torch.tensor([[5.0],[2.0]])  # Target output

# Step 2: Initialize parameters (weight and bias)
w = torch.tensor([[1.0], [1.0], [1.0]], requires_grad=True)  # Weight for 3 features
b = torch.tensor([[0.0]], requires_grad=True)  # Bias

# Step 3: Define the learning rate
learning_rate = 0.1

# Step 4: Forward pass
y_pred = x @ w + b  # Linear model: y = wx + b

# Step 5: Calculate the loss (Mean Squared Error)
loss = 0.5 * (y_pred - y_true).pow(2).mean()  # MSE loss
y_pred.shape, y_true.shape, loss,loss.shape, type((y_pred - y_true).pow(2))

(torch.Size([2, 1]),
 torch.Size([2, 1]),
 tensor(7.5400, grad_fn=<MulBackward0>),
 torch.Size([]),
 torch.Tensor)

In [77]:

# Step 6: Backward pass
loss.backward()  # Compute gradients

# Step 7: Print gradients
print(f"Gradient of loss w.r.t weight: {w.grad}")  # Print the entire gradient tensor
print(f"Gradient of loss w.r.t bias: {b.grad.item()}")  # Print bias gradient as scalar

# Step 8: Update parameters (weights and bias)
with torch.no_grad():  # Disable gradient tracking for updates
    w -= learning_rate * w.grad  # Update weight
    b -= learning_rate * b.grad  # Update bias

# Step 9: Zero the gradients after updating
w.grad.zero_()
b.grad.zero_()

# Step 10: Print updated parameters
print(f"Updated weight: {w}")  # Print the entire weight tensor
print(f"Updated bias: {b.item()}")  # Print bias as scalar

y_pred.shape, y_true.shape, loss,loss.shape

Gradient of loss w.r.t weight: tensor([[ 5.0100],
        [ 7.4800],
        [10.4900]])
Gradient of loss w.r.t bias: 3.200000286102295
Updated weight: tensor([[ 0.4990],
        [ 0.2520],
        [-0.0490]], requires_grad=True)
Updated bias: -0.320000022649765


(torch.Size([2, 1]),
 torch.Size([2, 1]),
 tensor(7.5400, grad_fn=<MulBackward0>),
 torch.Size([]))

In [24]:
# 神经网络的极简例子

import torch
import torch.nn as nn
import torch.optim as optim

# 设置随机种子以确保结果可复现
torch.manual_seed(42)

# 创建模拟数据
# 10条数据，每条4个特征
X = torch.randn(10, 4)
# 创建目标值（这里我们假设是回归问题，每个样本对应一个目标值）
y = torch.randn(10, 1)

# 定义神经网络模型
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(4, 8)  # 输入层到隐藏层
        self.fc2 = nn.Linear(8, 1)  # 隐藏层到输出层
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 实例化模型、损失函数和优化器
model = SimpleNet()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
# printmodel.parameters()）
# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    # 前向传播
    outputs = model(X) # 前向传播 - 计算最后的输出值
    loss = criterion(outputs, y) # 根据损失计算函数去计算当前损失
    
    # 反向传播和优化
    optimizer.zero_grad() # 重置梯度为0
    loss.backward() # 反向传播 - 反向传播会计算每个权重的偏导数（权重对于结果的影响）
    optimizer.step() # 这里才是更新权重值
    
    # 打印每100个epoch的损失
    if (epoch+1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 测试模型
model.eval()
with torch.no_grad():
    test_output = model(X)
    print("预测值:")
    print(test_output)
    print("实际值:")
    print(y)
    print(test_output-y)

Epoch [2/10], Loss: 0.5890
Epoch [4/10], Loss: 0.5890
Epoch [6/10], Loss: 0.5890
Epoch [8/10], Loss: 0.5890
Epoch [10/10], Loss: 0.5890
预测值:
tensor([[-0.7204],
        [-0.8232],
        [-0.4771],
        [-0.2774],
        [-0.3740],
        [-0.2620],
        [-0.5669],
        [-0.4316],
        [-0.2425],
        [-0.7539]])
实际值:
tensor([[-0.0499],
        [ 0.5263],
        [-0.0085],
        [ 0.7291],
        [ 0.1331],
        [ 0.8640],
        [-1.0157],
        [-0.8887],
        [ 0.1498],
        [-0.2089]])
tensor([[-0.6705],
        [-1.3496],
        [-0.4686],
        [-1.0065],
        [-0.5072],
        [-1.1260],
        [ 0.4488],
        [ 0.4572],
        [-0.3923],
        [-0.5450]])
